<a href="https://colab.research.google.com/github/konderal333/HGT-2022-EmDomArDon/blob/main/bert2bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load models and data

In [1]:
!git clone https://github.com/konderal333/HGT-2022-EmDomArDon.git

fatal: destination path 'HGT-2022-EmDomArDon' already exists and is not an empty directory.


In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
df = pd.read_csv('/content/HGT-2022-EmDomArDon/cikkek_10k_cleanedv1.csv')

In [4]:
print(df.isnull().values.any())
print(sum(df.duplicated()))

True
87


In [5]:
df = df.dropna()
df = df.drop_duplicates()

In [6]:
df = df.apply(lambda x: x.str.strip())
df['Textbody'] = df['Textbody'].apply(lambda x: re.sub(r"(\.)([A-Z])", r'\1 \2', str(x))) # add missing whitespace between sentences

df['Lead'] = df['Title'] + '. ' + df['Headline'] # maybe only headline and drop title?
df = df.drop(['Title', 'Headline'],axis=1)
df.head()

#reindexing the dataframe
df.reset_index(inplace=True)
df = df.drop(['index'], axis=1)

In [7]:
%%capture
!pip install transformers

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('SZTAKI-HLT/hubert-base-cc')

In [9]:
# bert2bert
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jVivzwtgoZx38yMeZA5BthGGvoPxBZk5' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1jVivzwtgoZx38yMeZA5BthGGvoPxBZk5" -O model_bert2bert.tar.gz && rm -rf /tmp/cookies.txt

# model
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1GEdOmmKZEJOD2Ei28FtVEf534-nBEcBd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1GEdOmmKZEJOD2Ei28FtVEf534-nBEcBd" -O model.tar.gz && rm -rf /tmp/cookies.txt

--2022-11-26 14:18:28--  https://docs.google.com/uc?export=download&confirm=t&id=1jVivzwtgoZx38yMeZA5BthGGvoPxBZk5
Resolving docs.google.com (docs.google.com)... 74.125.24.113, 74.125.24.138, 74.125.24.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gec13dubek0hos7drgmdu9hmmlam25ac/1669472250000/03710960119062529382/*/1jVivzwtgoZx38yMeZA5BthGGvoPxBZk5?e=download&uuid=9a0cac21-7248-4ff7-89ad-6cbff68c517b [following]
--2022-11-26 14:18:29--  https://doc-0o-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gec13dubek0hos7drgmdu9hmmlam25ac/1669472250000/03710960119062529382/*/1jVivzwtgoZx38yMeZA5BthGGvoPxBZk5?e=download&uuid=9a0cac21-7248-4ff7-89ad-6cbff68c517b
Resolving doc-0o-18-docs.googleusercontent.com (doc-0o-18-docs.googleusercontent.com)... 142.250.4.132, 2404:

In [10]:
import shutil
shutil.unpack_archive('model.tar.gz', 'model')
shutil.unpack_archive('model_bert2bert.tar.gz', 'bert2bert')

In [11]:
import torch
from transformers import EncoderDecoderModel

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

bert2bert_model_path = '/content/bert2bert/model/pytorch_model.bin'
bert2bert_config = '/content/bert2bert/model/config.json'

model_path = '/content/model/model/pytorch_model.bin'
config = '/content/model/model/config.json'

bert2bert = EncoderDecoderModel.from_pretrained(bert2bert_model_path, config=bert2bert_config).to(device)
model = EncoderDecoderModel.from_pretrained(model_path, config=config).to(device)

Tutorial notebook:

https://colab.research.google.com/drive/1WIk2bxglElfZewOHboPFNj8H44_VAyKE?usp=sharing&fbclid=IwAR2-pdnudSutvgqIlUUg4NP1Q0wJO9NQbteg_wVL9n8gsS2yMTfMp9YfC6w#scrollTo=bYmdx-W1NAky

The majority of the codes found in that notebook is outdated, so we needed to make a couple of augmentations and changes in order to make them work in our environment.

##Function to handle larger inputs than 512 tokens

In [ ]:
import re
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 7.4 MB/s 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
dict_of_tokens = {} 
#this dict will contain the article title as keys and the tokenized lists from the function below as values

def text_to_tokens(art_title, x):      
  #insert the text to tokenize; first breaks text into sentences, then makes a list of lists with the i-th element being the tokenization of the i-th sentence
  #dictionary is being used for faster computation and easier accessibility
  sentence_list = []
  token_list = []

  x = re.sub(r"(\.)([A-Z])", r'\1 \2', x) # fixes missings whitespaces between sentences
 
  for sen in nltk.tokenize.sent_tokenize(x):
    sentence_list.append(sen)
    token_list.append(tokenizer.tokenize(sen))

  dict_of_tokens[art_title] = token_list

# making the BERT tokenized sentences from each article
for i in range(len(df.Lead)):
  text_to_tokens(df.Lead[i].strip(), df.Textbody[i])

Some basic statistics about the number of tokens

In [ ]:
import matplotlib.pyplot as plt
import statistics

number_of_tokens_all = []

for tokens in dict_of_tokens.values():
  length = 0
  for lists in tokens:
    length += len(lists)
  number_of_tokens_all.append(length)


In [ ]:
plt.hist(number_of_tokens_all, bins=100, density=False)
plt.title('Distribution of number of tokens generated from articles')
plt.show()

In [ ]:
print("Median of tokens: ", statistics.median(number_of_tokens_all))
print("Mean of tokens: ", statistics.mean(number_of_tokens_all))

In [ ]:
# number of articles with tokens more that 510:
print(np.sum(np.asarray(number_of_tokens_all) > 510))

## Process data

In [12]:
def process_data_to_model_inputs(batch):
        # Tokenize the input and target data
        inputs = tokenizer(batch['Textbody'], padding='max_length', truncation=True, max_length=512)
        outputs = tokenizer(batch['Lead'], padding='max_length', truncation=True, max_length=512)

        batch['input_ids'] = inputs.input_ids
        batch['attention_mask'] = inputs.attention_mask
        batch['decoder_input_ids'] = outputs.input_ids
        batch['decoder_attention_mask'] = outputs.attention_mask
        batch['labels'] = outputs.input_ids.copy()

        # batch['labels'] = [[-100 if token == tokenizer.pad_token_id else token for token in labels]
        #                    for labels in batch['labels']]
        batch['labels'] = [-100 if token == tokenizer.pad_token_id else token for token in batch['labels']]

        return batch

In [13]:
#number of articles
df.shape[0]

9386

In [14]:
# Full Data

# split is 70-15-15

train_data = df.iloc[0:int(np.floor(df.shape[0]*0.7))]
val_data = df.iloc[int(np.floor(df.shape[0]*0.7)):int(np.floor(df.shape[0]*0.85))]
test_data = df.iloc[int(np.floor(df.shape[0]*0.85)):]


#Partial Data
#train_data = df.iloc[0:20]
#val_data = df.iloc[20:30]
#test_data = df.iloc[30:40]

#train_data.reset_index(inplace=True)
#val_data.reset_index(inplace=True)
#test_data.reset_index(inplace=True)

#train_data = train_data.drop(['index'], axis=1)
#val_data = val_data.drop(['index'], axis=1)
#test_data = test_data.drop(['index'], axis=1)

In [15]:
#Transforming the train and validation data
train_data = train_data.apply(process_data_to_model_inputs, axis=1)
train_data = train_data.drop(['Textbody', 'Lead'], axis=1)

val_data = val_data.apply(process_data_to_model_inputs, axis=1)
val_data = val_data.drop(['Textbody', 'Lead'], axis=1)

In [16]:
train_data.head()

,input_ids,attention_mask,decoder_input_ids,decoder_attention_mask,labels
0,"[2, 2155, 7155, 2524, 4046, 4341, 15269, 14826...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 21086, 2018, 6480, 2045, 26204, 14350, 804...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 21086, 2018, 6480, 2045, 26204, 14350, 804..."
1,"[2, 30196, 9688, 11787, 11628, 2005, 11149, 22...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 28589, 6997, 2039, 5833, 6505, 2094, 2005,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 28589, 6997, 2039, 5833, 6505, 2094, 2005,..."
2,"[2, 4405, 2546, 8984, 3576, 2079, 2005, 16667,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 5388, 28306, 7996, 2005, 16667, 3021, 2653...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 5388, 28306, 7996, 2005, 16667, 3021, 2653..."
3,"[2, 22158, 6688, 29536, 2005, 13549, 31735, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 12930, 12887, 6704, 7671, 2837, 20664, 297...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 12930, 12887, 6704, 7671, 2837, 20664, 297..."
4,"[2, 10708, 11447, 2685, 2066, 9763, 31736, 200...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 2930, 29814, 2066, 9763, 2033, 2718, 31741...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 2930, 29814, 2066, 9763, 2033, 2718, 31741..."


In [17]:
%%capture
!pip install datasets

In [18]:
# converting to arrow dataset (easier to use)
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

train_data = Dataset(pa.Table.from_pandas(train_data))
val_data = Dataset(pa.Table.from_pandas(val_data))
test_data = Dataset(pa.Table.from_pandas(test_data))

# format type
train_data

Dataset({
    features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
    num_rows: 6570
})

In [19]:
#converting the train and validation sets to tensors
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [20]:
# example
train_data["labels"]

tensor([[    2, 21086,  2018,  ...,  -100,  -100,  -100],
        [    2, 28589,  6997,  ...,  -100,  -100,  -100],
        [    2,  5388, 28306,  ...,  -100,  -100,  -100],
        ...,
        [    2,  2155,  3165,  ...,  -100,  -100,  -100],
        [    2,  3512,  2664,  ...,  -100,  -100,  -100],
        [    2,  5753,  2336,  ...,  -100,  -100,  -100]])

## Prediction and evaluation before training

In [28]:
# Metrics
!pip install rouge_score
import datasets
rouge = datasets.load_metric("rouge")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [35]:
# Summary generation

def generate_summary(batch):
    # cut off at BERT max length 512
    inputs = tokenizer(batch['Textbody'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
    input_ids = inputs.input_ids.to('cuda')
    attention_mask = inputs.attention_mask.to('cuda')

    outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=50)

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch['pred_summary'] = output_str

    return batch


In [24]:
# Example article
batch = df.iloc[10]
batch['Textbody']

'Találkozni szeretett volna a Kölcsey Ferenc Gimnázium tanárait menesztő tankerületi vezetővel Soproni Tamás, terézvárosi polgármester. Marosi Betrix azonban nem látta indokoltnak, hogy beszélgessenek. A tankerületi vezető levelét Facebook-oldalán tette közzé a Momentum politikusa. Azt írja Marosi, hogy „természetesen a tankerületi központnak is az az érdeke, hogy az iskolában a helyzet normalizálódjon, a pedagógusok tanítsanak, a diákok tanuljanak”. A nevelés-oktatás feltételeinek biztosítása a VI. kerület állami fenntartású iskoláiban a tankerület feladata és felelőssége,„a tankerületi központ sem kíván beleszólni abba, hogy az Önkormányzat hogyan és kikkel látja el a feladatait.”A tankerületi vezető nem látja indokoltnak, hogy személyesen is találkozzon a kerület polgármesterével, mert a munkáltatói intézkedés „kizárólag a hatályos jogszabályokból fakadó lépés volt”. Marosi szerint az érintett pedagógusok a többszöri figyelmeztetés ellenére is visszatérően megszegték „a pedagógus jo

In [25]:
# Now the summary
inputs = tokenizer(batch['Textbody'], padding='max_length', truncation=True, max_length=512, return_tensors='pt')
input_ids = inputs.input_ids.to('cuda')
attention_mask = inputs.attention_mask.to('cuda')
outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=50)
output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
output_str[0]

'Soproni Tamás szerint a tankerületi központnak is az az érdeke, hogy a tanárok és diákok tanuljanak.'

In [26]:
# Original lead
df.iloc[10]['Lead']

'Lepattintotta a kölcseys tanárokat menesztő tankerületi vezető Soproni Tamást, aki beszélgetésre hívta. Nem akart találkozni a momentumos polgármesterrel, mert ő sem akar beleszólni abba, hogy az önkormányzat hogyan és kikkel látja el a feladatait.'

In [27]:
# let's see the tokens
for t in input_ids:
    print(tokenizer.convert_ids_to_tokens(t))

['[CLS]', 'Találkoz', '##ni', 'szeretett', 'volna', 'a', 'Kölcs', '##ey', 'Ferenc', 'Gimnázium', 'tanára', '##it', 'men', '##esztő', 'tank', '##erületi', 'vezető', '##vel', 'Sopron', '##i', 'Tamás', ',', 'ter', '##éz', '##városi', 'polgármester', '.', 'Maros', '##i', 'Bet', '##rix', 'azonban', 'nem', 'látta', 'indokolt', '##nak', ',', 'hogy', 'beszélg', '##essenek', '.', 'A', 'tank', '##erületi', 'vezető', 'level', '##ét', 'Facebook', '-', 'oldalán', 'tette', 'közzé', 'a', 'Mo', '##ment', '##um', 'politikus', '##a', '.', 'Azt', 'írja', 'Maros', '##i', ',', 'hogy', '„', 'természetesen', 'a', 'tank', '##erületi', 'központ', '##nak', 'is', 'az', 'az', 'érdeke', ',', 'hogy', 'az', 'iskolában', 'a', 'helyzet', 'norm', '##alizál', '##ódjon', ',', 'a', 'pedagógusok', 'tanít', '##sanak', ',', 'a', 'diákok', 'tanul', '##janak', '”', '.', 'A', 'nevelés', '-', 'oktatás', 'feltételeinek', 'biztosítása', 'a', 'VI', '.', 'kerület', 'állami', 'fenntart', '##ású', 'iskol', '##áiban', 'a', 'tank', '##e

In [28]:
# Now evaluate the whole test set
batch_size = 64

results = test_data.map(generate_summary, batched=True, batch_size=batch_size)
scores = rouge.compute(predictions=results["pred_summary"], references=results["Lead"], rouge_types=["rouge1", "rouge2", "rougeL"])
print(scores['rouge1'].mid)
print(scores['rouge2'].mid)
print(scores['rougeL'].mid)

  0%|          | 0/22 [00:00<?, ?ba/s]

Score(precision=0.14045474872072017, recall=0.07940448993799529, fmeasure=0.09689360803523799)

In [29]:
# let's see a couple examples
print(results['Lead'][3])
print(results['pred_summary'][3])
print("")
print(results['Lead'][5])
print(results['pred_summary'][5])
print("")
print(results['Lead'][9])
print(results['pred_summary'][9])

Elon Musk visszaengedné Donald Trumpot a Twitterre. A milliárdos hibának tartja a 2021-ben hozott döntést a volt amerikai elnök végleges tiltásáról.
Elon Musk szerint hiba volt, hogy nem volt helyes letiltani Donald Trump volt amerikai elnök fiókját.

Messit látni és beleszeretni a Troyes-ba. A PSG hétvégi meccsén a hangulat közelebb volt a Fővárosi Nagycirkuszban tapasztaltakhoz, Messi és Neymar labdaérintésénél sikongatott a nézőtér, a hazai góloknál tűzijáték volt és hivatásos...
A Paris Saint - Germain szurkolói nem csak a Paris Saint - Germain elleni meccsre járnak el, hanem a Paris Saint - Germain elleni meccsre is.

Kína járványkezelési stratégiája fenntarthatatlan, ráadásul emberjogi és gazdasági szempontból is aggályos a WHO szerint. A „dinamikus zéró-Covid” miatt van több mint egy hónapja vesztegzár alatt a 25 milliós Sanghaj.
A WHO szerint a kínai járványkezelés fenntartása fenntarthatatlan.


As we can see, the sentences are mostly correct grammatically, but their meaning leaves a lot to be desired.

## Train further

In [21]:
#already default
#model.config.decoder_start_token_id == tokenizer.cls_token_id
#model.config.eos_token_id == tokenizer.sep_token_id
#model.config.pad_token_id == tokenizer.pad_token_id
#model.config.vocab_size == model.config.encoder.vocab_size

In [22]:
#még nem akarom állítgatni
#model.config.max_length = 142
#model.config.min_length = 56
#model.config.no_repeat_ngram_size = 3
#model.config.early_stopping = True
#model.config.length_penalty = 2.0
#model.config.num_beams = 4

In [23]:
%%capture
!rm seq2seq_trainer.py
!rm seq2seq_training_args.py
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_training_args.py

In [24]:
%%capture
!pip install git-python==1.0.3
!pip install rouge_score
!pip install sacrebleu

In [25]:
from seq2seq_trainer import Seq2SeqTrainer
from seq2seq_training_args import Seq2SeqTrainingArguments

In [52]:
# original training args
training_args = torch.load('/content/model/model/training_args.bin')
training_args

Seq2SeqTrainingArguments(output_dir='/mnt/idms/home/botondbarta/trainers_wo_nlc', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=13, per_device_eval_batch_size=13, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, learning_rate=5e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=8000, log_level=-1, log_level_replica=-1, log_on_each_node=True, logging_dir='/mnt/idms/home/botondbarta/trainers_wo_nlc/runs/Oct29_18-17-26_gpu100.ilab.sztaki.hu', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=500, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS:

In [50]:
batch_size = 16

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True, 
    output_dir="./output",
    logging_steps=16,
    save_steps=10,
    eval_steps=16,
    gradient_accumulation_steps=1, 
    eval_accumulation_steps=None, 
    eval_delay=0, 
    learning_rate=5e-05, 
    weight_decay=0.01, 
    adam_beta1=0.9, 
    adam_beta2=0.999, 
    adam_epsilon=1e-08, 
    max_grad_norm=1.0, 
)




In [30]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [31]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

Using cuda_amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6570
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1233
  Number of trainable parameters = 249636609
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
16,3.263400,2.000068,0.000400,0.000200,0.000300
32,0.276000,0.195544,0.000000,0.000100,0.000000
48,0.166200,0.163389,0.001200,0.001200,0.001100
64,0.147600,0.144132,0.000500,0.000500,0.000500
80,0.064500,0.020353,0.000600,0.000500,0.000500
96,0.030300,0.004106,0.000300,0.000300,0.000300
112,0.027600,0.021213,0.001400,0.001100,0.001200
128,0.027500,0.040037,0.001100,0.000800,0.000800


Saving model checkpoint to ./output/checkpoint-10
Configuration saved in ./output/checkpoint-10/config.json
Model weights saved in ./output/checkpoint-10/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
***** Running Evaluation *****
  Num examples = 1408
  Batch size = 16
Saving model checkpoint to ./output/checkpoint-20
Configuration saved in ./output/checkpoint-20/config.json
Model weights saved in ./output/checkpoint-20/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/tra

KeyboardInterrupt: ignored

In [37]:
# load the partially trained model
model = EncoderDecoderModel.from_pretrained("/content/output/checkpoint-140").to(device)

loading configuration file /content/output/checkpoint-140/config.json
Model config EncoderDecoderConfig {
  "_commit_hash": null,
  "_name_or_path": "/content/model/model/pytorch_model.bin",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "SZTAKI-HLT/hubert-base-cc",
    "add_cross_attention": true,
    "architectures": null,
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hid

## Evaluation after further training

In [40]:
batch_size = 64

results = test_data.map(generate_summary, batched=True, batch_size=batch_size)
scores = rouge.compute(predictions=results["pred_summary"], references=results["Lead"], rouge_types=["rouge1", "rouge2", "rougeL"])
print(scores['rouge1'].mid)
print(scores['rouge2'].mid)
print(scores['rougeL'].mid)

  0%|          | 0/22 [00:00<?, ?ba/s]

Score(precision=0.009718281248130407, recall=0.016238665502035506, fmeasure=0.01157736288744703)

In [43]:
# save to csv
results_df = pd.DataFrame({})
results_df['Lead'] = results['Lead']
results_df['pred_summary'] = results['pred_summary']
results_df['Textbody'] = results['Textbody']
results_df.to_csv('results_df.csv', index=False)

In [49]:
results_df['pred_summary'].iloc[40] #lmao

'Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák Novák'